In [82]:
import calendar
import gc
import os
import sys
import json

import boto3
from botocore.exceptions import ClientError
from dotenv import load_dotenv
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandasql as ps
import scipy
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from implicit.als import AlternatingLeastSquares
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from implicit.als import AlternatingLeastSquares
import zipfile


In [84]:
sys.path.append('/home/mle-user/mle_projects/mle-pr-final/')
from src.helpers import load_config_to_globals, start_mlflow_server

In [85]:
load_config_to_globals('config.json')

In [86]:
requirements_path

'/home/mle-user/mle_projects/mle-dvc/requirements.txt'

In [62]:
os.listdir(os.getcwd())

['eda.ipynb', 'config.json']

In [3]:
os.listdir('/home/mle-user/mle_projects/mle-pr-final/data')

['category_tree.csv',
 'item_properties_part1.csv',
 'events.csv',
 'item_properties_part2.csv',
 'archive.zip']

In [4]:
arch_path = '/home/mle-user/mle_projects/mle-pr-final/data'
arch_name ='archive.zip'
dest = '/home/mle-user/mle_projects/mle-pr-final/data'

In [5]:
data_path = '/home/mle-user/mle_projects/mle-pr-final/data'

In [6]:
if len(os.listdir(data_path))==1:
     zip_path = os.path.join(arch_path, arch_name)
     with zipfile.ZipFile(zip_path, 'r') as zip_ref:
          zip_ref.extractall(dest)
else:
     pass

In [7]:
os.listdir(data_path)

['category_tree.csv',
 'item_properties_part1.csv',
 'events.csv',
 'item_properties_part2.csv',
 'archive.zip']

In [8]:
category_tree = pd.read_csv(os.path.join(data_path, 'category_tree.csv'))
events = pd.read_csv(os.path.join(data_path, 'events.csv'))
item_properties_part1 = pd.read_csv(os.path.join(data_path, 'item_properties_part1.csv'))
item_properties_part2 = pd.read_csv(os.path.join(data_path, 'item_properties_part2.csv'))

In [9]:
def explore_data(df):
    """
    Исследует структуру данных: типы данных, пропущенные значения, базовые статистики.
    """
    print("===== Основная информация о данных =====")
    print(f"Размер датасета: {df.shape}")
    print("\nТипы данных:")
    print(df.dtypes)
    
    print("\nПропущенные значения:")
    missing_values = df.isnull().sum()
    print(missing_values[missing_values > 0])
    
    print("\nБазовая статистика:")
    print(df.describe())
    print(df.head(3))

In [10]:
explore_data(category_tree)

===== Основная информация о данных =====
Размер датасета: (1669, 2)

Типы данных:
categoryid      int64
parentid      float64
dtype: object

Пропущенные значения:
parentid    25
dtype: int64

Базовая статистика:
        categoryid     parentid
count  1669.000000  1644.000000
mean    849.285201   847.571168
std     490.195116   505.058485
min       0.000000     8.000000
25%     427.000000   381.000000
50%     848.000000   866.000000
75%    1273.000000  1291.000000
max    1698.000000  1698.000000
   categoryid  parentid
0        1016     213.0
1         809     169.0
2         570       9.0


In [11]:
explore_data(events)

===== Основная информация о данных =====
Размер датасета: (2756101, 5)

Типы данных:
timestamp          int64
visitorid          int64
event             object
itemid             int64
transactionid    float64
dtype: object

Пропущенные значения:
transactionid    2733644
dtype: int64

Базовая статистика:
          timestamp     visitorid        itemid  transactionid
count  2.756101e+06  2.756101e+06  2.756101e+06   22457.000000
mean   1.436424e+12  7.019229e+05  2.349225e+05    8826.497796
std    3.366312e+09  4.056875e+05  1.341954e+05    5098.996290
min    1.430622e+12  0.000000e+00  3.000000e+00       0.000000
25%    1.433478e+12  3.505660e+05  1.181200e+05    4411.000000
50%    1.436453e+12  7.020600e+05  2.360670e+05    8813.000000
75%    1.439225e+12  1.053437e+06  3.507150e+05   13224.000000
max    1.442545e+12  1.407579e+06  4.668670e+05   17671.000000
       timestamp  visitorid event  itemid  transactionid
0  1433221332117     257597  view  355908            NaN
1  1433224214

In [12]:
explore_data(item_properties_part1)

===== Основная информация о данных =====
Размер датасета: (10999999, 4)

Типы данных:
timestamp     int64
itemid        int64
property     object
value        object
dtype: object

Пропущенные значения:
Series([], dtype: int64)

Базовая статистика:
          timestamp        itemid
count  1.100000e+07  1.100000e+07
mean   1.435158e+12  2.333851e+05
std    3.327653e+09  1.348258e+05
min    1.431227e+12  0.000000e+00
25%    1.432436e+12  1.165150e+05
50%    1.433646e+12  2.334990e+05
75%    1.437880e+12  3.501860e+05
max    1.442113e+12  4.668660e+05
       timestamp  itemid    property                            value
0  1435460400000  460429  categoryid                             1338
1  1441508400000  206783         888          1116713 960601 n277.200
2  1439089200000  395014         400  n552.000 639502 n720.000 424566


In [13]:
explore_data(item_properties_part2)

===== Основная информация о данных =====
Размер датасета: (9275903, 4)

Типы данных:
timestamp     int64
itemid        int64
property     object
value        object
dtype: object

Пропущенные значения:
Series([], dtype: int64)

Базовая статистика:
          timestamp        itemid
count  9.275903e+06  9.275903e+06
mean   1.435156e+12  2.333968e+05
std    3.327970e+09  1.348682e+05
min    1.431227e+12  0.000000e+00
25%    1.432436e+12  1.165175e+05
50%    1.433646e+12  2.334620e+05
75%    1.437880e+12  3.504470e+05
max    1.442113e+12  4.668660e+05
       timestamp  itemid property            value
0  1433041200000  183478      561           769062
1  1439694000000  132256      976  n26.400 1135780
2  1435460400000  420307      921  1149317 1257525


In [14]:
category_tree.categoryid.nunique(),category_tree.parentid.nunique(), category_tree.shape[0]

(1669, 362, 1669)

In [15]:
1669/362

4.610497237569061

### заметка

Количество категорий больше в 4,6 раза чем parentid, значит parentid это что то вроде группы, например как для треков был жанр.
Есть 26 категорий без parentid, присвоим им parentid равный 0, так как такого parentid в первоначальных данных нет.

In [16]:
events.head(3)

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN


In [17]:
events.event.value_counts()

event
view           2664312
addtocart        69332
transaction      22457
Name: count, dtype: int64

In [18]:
events.event.value_counts()/events.shape[0]

event
view           0.966696
addtocart      0.025156
transaction    0.008148
Name: count, dtype: float64

In [19]:
event_gr = events.groupby('event', as_index=False).agg({'visitorid': 'nunique', 'itemid': 'nunique'})

In [20]:
event_gr

,event,visitorid,itemid
0,addtocart,37722,23903
1,transaction,11719,12025
2,view,1404179,234838


In [21]:
event_gr['visitor_share'] = event_gr['visitorid'] / events['visitorid'].nunique()
event_gr['item_share'] = event_gr['itemid'] / events['itemid'].nunique()

In [22]:
event_gr

,event,visitorid,itemid,visitor_share,item_share
0,addtocart,37722,23903,0.026799,0.101688
1,transaction,11719,12025,0.008326,0.051157
2,view,1404179,234838,0.997584,0.999051


In [23]:
events[~events.transactionid.isna()].head(3)

,timestamp,visitorid,event,itemid,transactionid
130,1433222276276,599528,transaction,356475,4000.0
304,1433193500981,121688,transaction,15335,11117.0
418,1433193915008,552148,transaction,81345,5444.0


In [24]:
events[events.transactionid.isna()].shape[0], events.shape[0]

(2733644, 2756101)

In [25]:
item_properties_part1.head(3)

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566


In [26]:
item_properties_part1[item_properties_part1['itemid'] == 460429]

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
2122725,1439694000000,460429,202,692224
2595719,1432436400000,460429,839,963847
6332062,1431226800000,460429,917,692224
6490575,1431226800000,460429,364,1058790
8129781,1431226800000,460429,available,0
8368211,1431226800000,460429,283,1103756 9705 963847
8566995,1433646000000,460429,283,1103756 9705 963847
8752572,1431226800000,460429,6,9705
8753668,1431226800000,460429,776,674847


In [27]:
item_properties_part2[item_properties_part2['itemid'] == 460429]

,timestamp,itemid,property,value
151187,1438484400000,460429,283,9705 963847 692224
349970,1442113200000,460429,283,9705 963847 692224
548753,1439694000000,460429,283,9705 963847 692224
563950,1431226800000,460429,888,692224
747536,1436065200000,460429,283,9705 963847 692224
788083,1435460400000,460429,764,1285872
1266667,1435460400000,460429,227,1103756 9705
1465450,1431226800000,460429,227,1103756 9705
1664233,1431831600000,460429,227,1103756 9705
1863016,1433041200000,460429,227,1103756 9705


In [28]:
item_properties_part1.property.value_counts(ascending=False)[0:10]

property
888           1629817
790            970800
available      817387
categoryid     426305
6              343207
283            323681
776            311654
678            261829
364            256340
202            242984
Name: count, dtype: int64

In [29]:
item_properties_part2.head(3)

,timestamp,itemid,property,value
0,1433041200000,183478,561,769062
1,1439694000000,132256,976,n26.400 1135780
2,1435460400000,420307,921,1149317 1257525


In [30]:
category_tree[category_tree['categoryid'] == 1338]

,categoryid,parentid
742,1338,1278.0


In [31]:
category_tree[category_tree['parentid'] == 1278.0]

,categoryid,parentid
170,578,1278.0
742,1338,1278.0
1134,1374,1278.0


In [32]:
category_tree.head(3)

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0


In [33]:
categoryid_prop1 = item_properties_part1[item_properties_part1['property'] == 'categoryid']
categoryid_prop2 = item_properties_part2[item_properties_part2['property'] == 'categoryid']
is_available1 = item_properties_part1[item_properties_part1['property'] == 'available']
is_available2 = item_properties_part1[item_properties_part1['property'] == 'available']

In [34]:
is_available = pd.concat([is_available1, is_available2])
category_prop = pd.concat([categoryid_prop1, categoryid_prop2])

### Признак доступности довара разделим на две таблицы, когда был доступен и когда нет.

In [35]:
is_available 

,timestamp,itemid,property,value
5,1436065200000,285026,available,0
15,1437274800000,186518,available,0
79,1433646000000,423682,available,0
82,1434250800000,316253,available,1
96,1437274800000,430459,available,0
...,...,...,...,...
10999926,1431831600000,350051,available,0
10999933,1433646000000,358669,available,0
10999959,1442113200000,362293,available,1
10999986,1439089200000,259404,available,1


In [36]:
is_available.groupby('value', as_index=False).agg({'itemid':'nunique','timestamp': 'count'})

,value,itemid,timestamp
0,0,219932,939226
1,1,57829,695548


In [37]:
is_available['value'] = is_available['value'].astype(int)

In [38]:
is_available_true = is_available[is_available['value'] ==1]
is_available_false = is_available[is_available['value'] ==0]

In [39]:
is_available.shape[0], is_available_true.shape[0]+ is_available_false.shape[0]

(1634774, 1634774)

In [40]:
is_available_true.columns = ['timestamp_available_true', 'itemid', 'property', 'value']
is_available_false.columns  = ['timestamp_available_false', 'itemid', 'property', 'value']

In [41]:
is_available_true['timestamp_available_true_min'] = is_available_true['timestamp_available_true']
is_available_false['timestamp_available_false_min'] = is_available_false['timestamp_available_false']

/tmp/ipykernel_1868/1894990614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_available_true['timestamp_available_true_min'] = is_available_true['timestamp_available_true']
/tmp/ipykernel_1868/1894990614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  is_available_false['timestamp_available_false_min'] = is_available_false['timestamp_available_false']


In [42]:
is_available_true_gr = is_available_true.groupby('itemid', as_index=False).agg({'timestamp_available_true': 'max', 'timestamp_available_true_min': 'min'})

In [43]:
is_available_false_gr = is_available_false.groupby('itemid', as_index=False).agg({'timestamp_available_false': 'max', 'timestamp_available_false_min': 'min'})

In [44]:
events

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN
2756097,1438399813142,762376,view,115946,NaN
2756098,1438397820527,1251746,view,78144,NaN
2756099,1438398530703,1184451,view,283392,NaN


In [45]:
import psutil

print(psutil.virtual_memory())  # Информация о памяти
print(psutil.cpu_percent())  

svmem(total=33651253248, available=29518639104, percent=12.3, used=3641282560, free=26388152320, active=702472192, inactive=5241769984, buffers=550748160, cached=3071070208, shared=1171456, slab=1195773952)
8.2


In [46]:
import gc
del item_properties_part1
del item_properties_part2
del is_available
gc.collect()

0

In [47]:
merged = events.merge(is_available_true_gr,
    how='left',on='itemid')

In [48]:
merged.shape[0], events.shape[0]

(2756101, 2756101)

In [49]:
merged = merged.merge(is_available_false_gr,how='left', on='itemid')

In [50]:
merged 

,timestamp,visitorid,event,itemid,transactionid,timestamp_available_true,timestamp_available_true_min,timestamp_available_false,timestamp_available_false_min
0,1433221332117,257597,view,355908,NaN,1.440904e+12,1.431832e+12,NaN,NaN
1,1433224214164,992329,view,248676,NaN,1.431227e+12,1.431227e+12,NaN,NaN
2,1433221999827,111016,view,318965,NaN,NaN,NaN,NaN,NaN
3,1433221955914,483717,view,253185,NaN,NaN,NaN,NaN,NaN
4,1433221337106,951259,view,367447,NaN,1.440904e+12,1.431227e+12,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2756096,1438398785939,591435,view,261427,NaN,NaN,NaN,1.433041e+12,1.433041e+12
2756097,1438399813142,762376,view,115946,NaN,NaN,NaN,1.431227e+12,1.431227e+12
2756098,1438397820527,1251746,view,78144,NaN,1.442113e+12,1.431227e+12,1.441508e+12,1.440904e+12
2756099,1438398530703,1184451,view,283392,NaN,NaN,NaN,NaN,NaN


In [51]:
del events
del is_available_true 
del is_available_false
gc.collect()

0

In [52]:
category_tree['categoryid'] = category_tree['categoryid'].astype(str)

In [53]:
category_prop = category_prop.merge(category_tree, how='left', left_on='value', right_on='categoryid')

In [54]:
category_prop['parentid'] = category_prop['parentid'].fillna(0).astype(int)

In [55]:
category_prop_gr = category_prop.groupby('itemid', as_index=False).parentid.unique()

In [56]:
category_prop_gr

,itemid,parentid
0,0,[293]
1,1,[113]
2,2,[1214]
3,3,[938]
4,4,[1174]
...,...,...
417048,466862,[480]
417049,466863,[788]
417050,466864,[340]
417051,466865,[1424]


In [57]:
merged = merged.merge(category_prop_gr,
    how='left', on='itemid')

In [58]:
merged = merged.fillna(0)

In [59]:
cols_to_tr = ['timestamp', 'visitorid', 'itemid', 'transactionid',
       'timestamp_available_true', 'timestamp_available_true_min',
       'timestamp_available_false', 'timestamp_available_false_min']

In [60]:
for c in cols_to_tr:
    merged[c] = merged[c].astype(int)


In [61]:
merged[merged['transactionid']>0]

,timestamp,visitorid,event,itemid,transactionid,timestamp_available_true,timestamp_available_true_min,timestamp_available_false,timestamp_available_false_min,parentid
130,1433222276276,599528,transaction,356475,4000,0,0,0,0,[1095]
304,1433193500981,121688,transaction,15335,11117,0,0,0,0,[897]
418,1433193915008,552148,transaction,81345,5444,0,0,0,0,[1383]
814,1433176736375,102019,transaction,150318,13556,0,0,0,0,[1141]
843,1433174518180,189384,transaction,310791,7244,1431226800000,1431226800000,0,0,[92]
...,...,...,...,...,...,...,...,...,...,...
2755294,1438377176570,1050575,transaction,31640,8354,0,0,0,0,[1637]
2755349,1438379878779,861299,transaction,456602,3643,1441508400000,1432436400000,1442113200000,1431226800000,[805]
2755508,1438357730123,855941,transaction,235771,4385,1442113200000,1432436400000,1440903600000,1431226800000,[805]
2755603,1438355560300,548772,transaction,29167,13872,1440903600000,1433646000000,0,0,[955]
